# Exercise Day 5

Design your own transfer function to highlight relevant features in the data set "wind.vti". Since you already know how to use the vtkArrayCalculator to compute new arrays, feel free to explore what other interesting data can you derive from the available data.

## Visualization 1: Isosurfaces

In [2]:
import vtk

In [37]:
# Read the file (to test that it was written correctly)
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("../data/wind_image.vti")
reader.Update()

# Convert the image to a polydata
imageDataGeometryFilter = vtk.vtkImageDataGeometryFilter()
imageDataGeometryFilter.SetInputConnection(reader.GetOutputPort())
imageDataGeometryFilter.Update()

scalarRange = reader.GetOutput().GetPointData().GetScalars().GetRange(-1)
contoursFilter = vtk.vtkContourFilter()
contoursFilter.SetInputConnection(reader.GetOutputPort())
contoursFilter.GenerateValues(15, scalarRange)

contoursMapper = vtk.vtkPolyDataMapper()
contoursMapper.SetInputConnection(contoursFilter.GetOutputPort())
contoursMapper.SetColorModeToMapScalars()
contoursMapper.ScalarVisibilityOn()
contoursMapper.SelectColorArray("wind_speed")
contoursMapper.SetScalarRange(scalarRange)

contoursActor = vtk.vtkActor()
contoursActor.SetMapper(contoursMapper)

actor = vtk.vtkActor()
actor.SetMapper(contoursMapper)
 
# Setup rendering
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(1,1,1)
renderer.ResetCamera()
 
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
 
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
 
renderWindowInteractor.SetRenderWindow(renderWindow)
renderWindowInteractor.Start()

## Visualization 2: Points

In [38]:
# Read the file (to test that it was written correctly)
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("../data/wind_image.vti")
reader.Update()

# Convert the image to a polydata
imageDataGeometryFilter = vtk.vtkImageDataGeometryFilter()
imageDataGeometryFilter.SetInputConnection(reader.GetOutputPort())
imageDataGeometryFilter.Update()

scalarRange = reader.GetOutput().GetPointData().GetScalars().GetRange(-1)
contoursFilter = vtk.vtkContourFilter()
contoursFilter.SetInputConnection(reader.GetOutputPort())
contoursFilter.GenerateValues(100, scalarRange)

contoursMapper = vtk.vtkPolyDataMapper()
contoursMapper.SetInputConnection(imageDataGeometryFilter.GetOutputPort())
contoursMapper.SetColorModeToMapScalars()
contoursMapper.ScalarVisibilityOn()
contoursMapper.SelectColorArray("wind_speed")
contoursMapper.SetScalarRange(scalarRange)

contoursActor = vtk.vtkActor()
contoursActor.SetMapper(contoursMapper)

actor = vtk.vtkActor()
actor.SetMapper(contoursMapper)
 
# Setup rendering
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(1,1,1)
renderer.ResetCamera()
 
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
 
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
 
renderWindowInteractor.SetRenderWindow(renderWindow)
renderWindowInteractor.Start()

## Visualization 3: Transfer function with ArrayCalculator

As the magnitude of the vector of wind velocity increases, the shade of red becomes darker. I assigned lower values of opacity to the outer layers, that were the ones with higher magnitudes in order to visualize the inner layers easier.

In [39]:
import vtk

#------------READER ----------------------
rectGridReader = vtk.vtkXMLImageDataReader()
rectGridReader.SetFileName("../data/wind_image.vti")
rectGridReader.Update()
#------------END READER ------------------

imageDataGeometryFilter = vtk.vtkImageDataGeometryFilter()
imageDataGeometryFilter.SetInputConnection(rectGridReader.GetOutputPort())
imageDataGeometryFilter.Update()

scalarRange = rectGridReader.GetOutput().GetPointData().GetScalars().GetRange(-1)
contoursFilter = vtk.vtkContourFilter()
contoursFilter.SetInputConnection(imageDataGeometryFilter.GetOutputPort())
contoursFilter.GenerateValues(30, scalarRange)

#------------ FILTER: CALCULATE VECTOR MAGNITUDE ----------------------
magnitudeCalcFilter = vtk.vtkArrayCalculator()
magnitudeCalcFilter.SetInputConnection(rectGridReader.GetOutputPort())
magnitudeCalcFilter.AddVectorArrayName('wind_velocity')
magnitudeCalcFilter.SetResultArrayName('magnitude')
magnitudeCalcFilter.SetFunction("mag(wind_velocity)") 
magnitudeCalcFilter.Update()
#------------END CALCULATE VECTOR MAGNITUDE ----------------------

#------------FILTER: RECTILINEAR GRID TO IMAGE DATA-----------
bounds = magnitudeCalcFilter.GetOutput().GetBounds()
dimensions = magnitudeCalcFilter.GetOutput().GetDimensions()
origin = (bounds[0], bounds[2], bounds[4])
spacing = ( (bounds[1]-bounds[0])/dimensions[0], 
            (bounds[3]-bounds[2])/dimensions[1],
            (bounds[5]-bounds[4])/dimensions[2])

imageData = vtk.vtkImageData()
imageData.SetOrigin(origin)
imageData.SetDimensions(dimensions)
imageData.SetSpacing(spacing)

probeFilter = vtk.vtkProbeFilter()
probeFilter.SetInputData(imageData)
probeFilter.SetSourceData(magnitudeCalcFilter.GetOutput())
probeFilter.Update()

imageData2 = probeFilter.GetImageDataOutput()
#------------END RECTILINEAR GRID TO IMAGE DATA-----------

##------------FILTER, MAPPER, AND ACTOR: VOLUME RENDERING -------------------
# Create transfer mapping scalar value to opacity
opacityTransferFunction = vtk.vtkPiecewiseFunction()
opacityTransferFunction.AddPoint(0, 0.02)
opacityTransferFunction.AddPoint(1, 0.05)
opacityTransferFunction.AddPoint(2, 0.1)
opacityTransferFunction.AddPoint(6, 0.1)
opacityTransferFunction.AddPoint(10, 0.1)
opacityTransferFunction.AddPoint(15, 0.1)
opacityTransferFunction.AddPoint(20, 0.3)
opacityTransferFunction.AddPoint(25, 0.3)
opacityTransferFunction.AddPoint(30, 0.3)
opacityTransferFunction.AddPoint(40, 0.3)
opacityTransferFunction.AddPoint(50, 0.7)
opacityTransferFunction.AddPoint(60, 0.8)
opacityTransferFunction.AddPoint(70, 0.9)


# Create transfer mapping scalar value to color
colorTransferFunction = vtk.vtkColorTransferFunction()
colorTransferFunction.AddRGBPoint(0.0, 239/255, 207/255, 210/255)
colorTransferFunction.AddRGBPoint(1.0, 242/255, 150/255, 147/255)
colorTransferFunction.AddRGBPoint(2.0, 249/255, 129/255, 124/255)
colorTransferFunction.AddRGBPoint(6.0, 247/255, 99/255, 93/255)
colorTransferFunction.AddRGBPoint(10.0, 247/255, 75/255, 69/255)
colorTransferFunction.AddRGBPoint(15.0, 242/255, 50/255, 43/255)
colorTransferFunction.AddRGBPoint(20.0, 242/255, 33/255, 26/255)
colorTransferFunction.AddRGBPoint(25.0, 214/255, 31/255, 25/255)
colorTransferFunction.AddRGBPoint(30.0, 186/255, 31/255, 26/255)
colorTransferFunction.AddRGBPoint(40.0, 165/255, 21/255, 16/255)
colorTransferFunction.AddRGBPoint(50.0, 104/255, 5/255, 2/255)
colorTransferFunction.AddRGBPoint(60.0, 89/255, 5/255, 2/255)
colorTransferFunction.AddRGBPoint(70.0, 58/255, 3/255, 1/255)


# The property describes how the data will look
volumeProperty = vtk.vtkVolumeProperty()
volumeProperty.SetColor(colorTransferFunction)
volumeProperty.SetScalarOpacity(opacityTransferFunction)
volumeProperty.ShadeOff()
volumeProperty.SetInterpolationTypeToLinear()


# The mapper / ray cast function know how to render the data
volumeMapper = vtk.vtkGPUVolumeRayCastMapper()
volumeMapper.SetInputData(imageData2)

# The volume holds the mapper and the property and
# can be used to position/orient the volume
volume = vtk.vtkVolume()
volume.SetMapper(volumeMapper)
volume.SetProperty(volumeProperty)

##------------END VOLUME RENDERING ----------------------

#---------RENDERER, RENDER WINDOW, AND INTERACTOR ----------
renderer = vtk.vtkRenderer()
renderer.SetBackground(0.5, 0.5, 0.5)
renderer.AddVolume(volume)
renderer.ResetCamera()

renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindow.SetSize(500, 500)
renderWindow.Render()

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renderWindow)
iren.Start()
#---------END RENDERER, RENDER WINDOW, AND INTERACTOR -------

In [10]:
%qtconsole